# Greek Letters

Option Greeks measure the exposure of option price or option delta to movement of different factors such as the underlying price, time and volatility.

**First Order Greeks**

Delta, Vega, Theta, Rho


**Second Order Greeks**
Gamma


In [ ]:
# import library
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
import pandas_datareader.data as web

*   C = call option price
*   N = CDF of the normal distribution
*   St= spot price of an asset
*   K = strike price
*   r = risk-free interest rate
*   t = time to maturity
*   σ = volatility of the asset

In [ ]:
"""
Black Scholes Model
"""
def d1(S, K, t, r, sigma):
    return(log(S / K) + (r + sigma ** 2 / 2.) * t) / sigma * sqrt(t)

def d2(S, K, t, r, sigma):
    return d1(S, K, t, r, sigma) - sigma * sqrt(t)

def bs_call(S, K, t, r, sigma):
    return S * norm.cdf(d1(S, K, t, r, sigma)) - K * exp(-r * t) * norm.cdf(d2(S, K, t, r, sigma))
  
def bs_put(S, K, t, r, sigma):
    return K * exp(-r * t) * norm.cdf(-d2(S, K, t, r, sigma)) - S * norm.cdf(-d1(S, K, t, r, sigma))

## Delta

Delta is the rate of change of the option price with respect to the price of the underlying asset.

It measures the first-order sensitivity of the price to a movement in stock price S.


In [28]:
def delta(Type, S, K, t, r, sigma):
  if Type == 'c':
    return norm.cdf(d1(S, K, t, r, sigma))
  elif Type == 'p':
    return -norm.cdf(-d1(S ,K, t, r, sigma))


# Gamma

Gamma is the rate of change of the portfolio's delta with respect to the underlying asset's price. It represents the second-order sensitivity of the option to a movement in the underlying asset’s price.

In [ ]:
def gamma(Type, S, K, t, r, sigma):
  if Type == 'c' or Type == 'p':
    return norm.pdf(d1(S, K, t, r, sigma)) / (S * sigma * sqrt(t))


# Vega

The Vega is the rate of change in the value of the option with respect to the volatility of the underlying asset.

In [ ]:
def vega(Type, S, K, t, r, sigma):
  if Type == 'c' or Type == 'p':
    return 0.01 * S * norm.pdf(d1(S, K, t, r, sigma) * sqrt(t))


# Theta

Theta is the rate of change of the value of the option with respect to the passage of time. It is also referred to as the time decay of the portfolio.

In [ ]:
def theta(Type, S, K, t, r, sigma):
  if Type == 'c':
    return 0.01 * (-(S * norm.pdf(d1(S, K, t, r, sigma)) * sigma) / (2 * sqrt(t)) - r * K * exp(-r * t) * norm.cdf(d2(S, K, t, r, sigma)))
  elif Type == 'p':
    return 0.01 * (-(S * norm.pdf(d1(S, K, t, r, sigma)) * sigma) / (2 * sqrt(t)) + r * K * exp(-r * t) * norm.cdf(-d2(S, K, t, r, sigma)))


# Rho

Rho is the rate of change of the value of a derivative with respect to the interest rate.

In [ ]:
def rho(Type, S, K, t, r, sigma):
  if Type == 'c':
    return 0.01 * (K * t * exp(-r * t) * norm.cdf(d2(S, K, t, r, sigma)))
  elif Type== 'p':
     return 0.01 * (-K * t * exp(-r * t) * norm.cdf(-d2(S, K, t, r, sigma)))

# Implied Volatility
It is defined as the expected future volatility of the stock over the life of the option. It is directly influenced by the supply and demand of the underlying option and the market’s expectation of the stock price’s direction.

In [21]:
def call_implied_volatility(Price, S, K, t, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = S * \
            norm.cdf(d1(S, K, t, r, sigma)) - K * exp(-r * t) * \
            norm.cdf(d2(S, K, t, r, sigma))
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

def put_implied_volatility(Price, S, K, t, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = K * exp(-r * t) - S + bs_call(S, K, t, r, sigma)
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

We are able to calculate the sigma value, volatility of the stock, by multiplying the standard deviation of the stock returns over the past year by the square root of 252 (number of days the market is open over a year). 

Current Stock price: last close price
Risk-free rate: the 10-year U.S. treasury yield which you could get from ^TNX

In [82]:
def previous_time():
  #today = datetime.now()
  today_raw = '05-26-2021'
  today = datetime.strptime(today_raw, "%m-%d-%Y")

  # time to maturity
  t = (datetime.strptime(expiry, "%m-%d-%Y") - today).days / 365
  one_day_ago = today.replace(day = today.day - 1)
  one_year_ago = today.replace(year = today.year - 1)

  return [one_day_ago, one_year_ago]

one_day_ago = previous_time()[0]
one_year_ago = previous_time()[1]

In [105]:
def optionPrice(stock, expiry, K):

  today = datetime.now()
  #today_raw = '12-18-2021'
  today = datetime.strptime(today_raw, "%m-%d-%Y")

  # Time to maturity
  t = (datetime.strptime(expiry, "%m-%d-%Y") - today).days / 365
  one_day_ago = today.replace(day = today.day - 1)
  one_year_ago = today.replace(year = today.year - 1)

  # Get data
  df = web.DataReader(stock, 'yahoo', one_year_ago, today)
  df = df.sort_values(by = "Date")
  df = df.dropna()

  # Underlying price
  S = df['Close'].iloc[-1]

  # Previous day's price
  df = df.assign(close_day_before=df.Close.shift(1))

  # Stock Return
  df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)

  # Sigma
  sigma = np.sqrt(252) * df['returns'].std()
  print("Sigma: ", sigma)

  # Risk free rate
  r = web.DataReader("^TNX", 'yahoo', one_day_ago, today)['Close'].iloc[-1]
  print("Risk free rate: ", r)

  print('The Option Price for ' + stock + ' at price ' + str(K) + ' on ' + expiry + ' is: ', bs_call(S, K, t, r, sigma))
  print("Implied Volatility: " +
        str(100 * call_implied_volatility(bs_call(S, K, t, r, sigma), S, K, t, r)) + " %")
  print("Delta: " + str(delta('c', S, K, t, r, sigma)))
  print("Gamma: " + str(gamma('c', S, K, t, r, sigma)))


In [106]:
# Expriation date
expiry = '06-04-2021'
# Strike price
K = 150

print(optionPrice('IBM', expiry, 150))
print(optionPrice('MSFT', expiry, 260))

Sigma:  0.2787305738058885
Risk free rate:  1.5640000104904175
The Option Price for IBM at price 150 on 06-04-2021 is:  2.2517731627012836
Implied Volatility: 27.90000000000002 %
Delta: 0.49937986378924215
Gamma: 0.06339010138071849
None
Sigma:  0.27862813024366295
Risk free rate:  1.5640000104904175
The Option Price for MSFT at price 260 on 06-04-2021 is:  5.1459075147165265
Implied Volatility: 27.90000000000002 %
Delta: 0.5016091796504965
Gamma: 0.036223384395352276
None
